# Data Wrangling - Iteration 1 - On Your Toes


Date : 23 April 2020
Version : 2.0

Data Sets analysed:


* Crime Data : Data Tables - LGA Criminal Incidents Visualisation - year ending December 2019

In this notebook, Crime datasets used in the On Your Toes - The fitness app are put through data wrangling tasks to inspect data, clean up and create custom datasets to be exported to the MySQL database.


Detailed data plan : https://mahara.infotech.monash.edu.au/mahara/artefact/file/download.php?file=229847

-------------------------------------------------------------

#### Import Libraries and Packages

In [1]:
import pandas as pd
import numpy as np

------------------------------------------------------

## Crime Data

### Dataset Info

Name: Data Tables - LGA Criminal Incidents Visualisation - year ending December 2019

Description: Table 03 Criminal incidents by principal offence, local government area and postcode or suburb/town - January 2010 to December 2019 

Link : https://www.crimestatistics.vic.gov.au/sites/default/files/embridge_cache/emshare/original/public/users/202003/e7/d73f8b607/Data_Tables_LGA_Recorded_Offences_Year_Ending_December_2019.xlsx

Permissions : CC BY 4.0

### Wrangling tasks


Read data into python and examine the column names

In [2]:
crime_df = pd.read_excel('Data_Tables_LGA_Criminal_Incidents_Year_Ending_December_2019.xlsx','Table 03')
print("Dataset shape : ",crime_df.shape)
print(crime_df.columns)

Dataset shape :  (309358, 9)
Index(['Year', 'Year ending', 'Local Government Area', 'Postcode',
       'Suburb/Town Name', 'Offence Division', 'Offence Subdivision',
       'Offence Subgroup', 'Incidents Recorded'],
      dtype='object')


Check dataset for null values

In [3]:
crime_df.isnull().sum()

Year                     0
Year ending              0
Local Government Area    0
Postcode                 0
Suburb/Town Name         0
Offence Division         0
Offence Subdivision      0
Offence Subgroup         0
Incidents Recorded       0
dtype: int64

Check dataset for duplicate values

In [4]:
print("Number of duplicate rows : ",len(crime_df[crime_df.duplicated(subset=None, keep='first')]))

Number of duplicate rows :  0


------------------------------------------------------------------------


Create "Suburb, Postcode" column for easy data filtering

In [5]:
# Convert columns to string data type
crime_df["Suburb/Town Name"] = crime_df["Suburb/Town Name"].astype(str)
crime_df["Postcode"]=crime_df["Postcode"].astype(str)

# Add new column "Suburb, Postcode" to the dataset
crime_df["SuburbPostcode"] = crime_df["Suburb/Town Name"] +", "+ crime_df["Postcode"]
crime_df.head(3)

,Year,Year ending,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,SuburbPostcode
0,2019,December,Alpine,3691,Dederang,B Property and deception offences,B30 Burglary/Break and enter,B321 Residential non-aggravated burglary,1,"Dederang, 3691"
1,2019,December,Alpine,3691,Dederang,E Justice procedures offences,E10 Justice procedures,E14 Pervert the course of justice or commit pe...,1,"Dederang, 3691"
2,2019,December,Alpine,3691,Dederang,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,1,"Dederang, 3691"


------------------------

Create a subset of the dataset that contains the local government area and the suburb from the crime_df dataset

In [6]:
# Create dataset and drop duplicate balues
lga_suburb = crime_df[['Local Government Area','SuburbPostcode']]

# Reset index
lga_suburb = lga_suburb.reset_index(drop=True)

# Rename "Local Government Area" column name as "LGA" for easier processing
lga_suburb.rename(columns={'Local Government Area':'LGA'}, inplace=True)

# Drop duplicates
lga_suburb= lga_suburb.drop_duplicates()
print("Length of lga_suburb dataset : ",len(lga_suburb))
lga_suburb.head(2)

Length of lga_suburb dataset :  3126


,LGA,SuburbPostcode
0,Alpine,"Dederang, 3691"
7,Alpine,"Glen Creek, 3691"


Export dataset as a CSV file

In [7]:
lga_suburb.to_csv("lga_suburb_export.csv", index=False)

Create "Total Incidents by Suburb/PC" column for data filtering purposes

In [8]:
# Create custom dataframe to store the total incident numbers grouped by "Suburb, Postcode column"
crime_sum=crime_df.groupby(['SuburbPostcode'])[['Incidents Recorded']].sum().reset_index()

# Create a new dataframe merging the crime_df and crime_sum to include "Total incidents by Suburb/PC"
crime_total_inc = pd.merge(crime_df, crime_sum, how='outer', on=['SuburbPostcode', 'SuburbPostcode'])
crime_total_inc.rename(columns={'Incidents Recorded_x':'Incidents Recorded','Incidents Recorded_y':'Total Incidents by Suburb/PC'}, inplace=True)

crime_total_inc.head(2)

,Year,Year ending,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,SuburbPostcode,Total Incidents by Suburb/PC
0,2019,December,Alpine,3691,Dederang,B Property and deception offences,B30 Burglary/Break and enter,B321 Residential non-aggravated burglary,1,"Dederang, 3691",59
1,2019,December,Alpine,3691,Dederang,E Justice procedures offences,E10 Justice procedures,E14 Pervert the course of justice or commit pe...,1,"Dederang, 3691",59


Create "Total incidents by Offence Division and Suburb, Postcode" column for filtering purposes

In [9]:
# Create custom dataframe to store the total incident numbers grouped by "Offense Division" and "SuburbPostcode" column
crime_div_sum=crime_total_inc.groupby(['Offence Division','SuburbPostcode'])[['Incidents Recorded']].sum().reset_index()

# Create a new dataframe merging the crime_df and crime_sum to include "Total incidents by Suburb/PC"
suburb_and_od = pd.merge(crime_total_inc, crime_div_sum, how='outer',left_on=['Offence Division','SuburbPostcode'], right_on = ['Offence Division','SuburbPostcode'])
suburb_and_od.rename(columns={'Incidents Recorded_x':'Incidents Recorded','Incidents Recorded_y':'Total Incidents by Offence Division and Suburb/PC'}, inplace=True)

suburb_and_od.head(2)

,Year,Year ending,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,SuburbPostcode,Total Incidents by Suburb/PC,Total Incidents by Offence Division and Suburb/PC
0,2019,December,Alpine,3691,Dederang,B Property and deception offences,B30 Burglary/Break and enter,B321 Residential non-aggravated burglary,1,"Dederang, 3691",59,20
1,2019,December,Alpine,3691,Dederang,B Property and deception offences,B20 Property damage,B21 Criminal damage,1,"Dederang, 3691",59,20


Create "Total incidents by Offence Subgroup and Suburb, Postcode" column for filtering purposes

In [10]:
# Create custom dataframe to store the total incident numbers grouped by "Offence Subgroup" and "SuburbPostcode"
crime_sub_sum=crime_total_inc.groupby(['Offence Subgroup','SuburbPostcode'])[['Incidents Recorded']].sum().reset_index()

# Create a new dataframe merging the crime_df and crime_sum to include "Total incidents by Suburb/PC"
crime_all_totals = pd.merge(suburb_and_od, crime_sub_sum, how='outer',left_on=['Offence Subgroup','SuburbPostcode'], right_on = ['Offence Subgroup','SuburbPostcode'])
crime_all_totals.rename(columns={'Incidents Recorded_x':'Incidents Recorded','Incidents Recorded_y':'Total Incidents by Offence Subgroup and Suburb/PC'}, inplace=True)

crime_all_totals.head(2)

,Year,Year ending,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,SuburbPostcode,Total Incidents by Suburb/PC,Total Incidents by Offence Division and Suburb/PC,Total Incidents by Offence Subgroup and Suburb/PC
0,2019,December,Alpine,3691,Dederang,B Property and deception offences,B30 Burglary/Break and enter,B321 Residential non-aggravated burglary,1,"Dederang, 3691",59,20,2
1,2014,December,Alpine,3691,Dederang,B Property and deception offences,B30 Burglary/Break and enter,B321 Residential non-aggravated burglary,1,"Dederang, 3691",59,20,2


Filter dataframe for year 2017 - 2019 as the app uses the last 3 years to calculate safety levels

In [11]:
crime_all_totals=crime_all_totals[crime_all_totals["Year"].isin([2017, 2018,2019])]
crime_all_totals.shape

(100833, 13)

-------------------------------------------------

Import the crime severity custom dataset as a dataframe

In [12]:
offence_severity_levels = pd.read_csv("Offence_Severity_Levels.csv")
print("Length of offence_severity_levels dataset", len(offence_severity_levels))

Length of offence_severity_levels dataset 27


Merge the offence_severity_levels dataset and crime_all_totals

In [13]:
crime_severity_added = pd.merge(crime_all_totals, offence_severity_levels, how='outer', on=['Offence Subgroup', 'Offence Subgroup'])
crime_severity_added.drop(crime_severity_added.tail(1).index,inplace=True)

# Since Missing values appear in the new dataset due to the outer join performed, replace missing values with 0
crime_severity_added["Rating"].fillna(0, inplace = True) 

crime_severity_added.head(2)

,Year,Year ending,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,SuburbPostcode,Total Incidents by Suburb/PC,Total Incidents by Offence Division and Suburb/PC,Total Incidents by Offence Subgroup and Suburb/PC,Rating
0,2019.0,December,Alpine,3691,Dederang,B Property and deception offences,B30 Burglary/Break and enter,B321 Residential non-aggravated burglary,1.0,"Dederang, 3691",59.0,20.0,2.0,0.0
1,2018.0,December,Alpine,3697,Tawonga,B Property and deception offences,B30 Burglary/Break and enter,B321 Residential non-aggravated burglary,1.0,"Tawonga, 3697",58.0,19.0,2.0,0.0


Create custom dataframe to store the total incident numbers grouped by "Rating" and "SuburbPostcode"

In [14]:
crime_sev_sub_sum=crime_severity_added.groupby(['SuburbPostcode','Rating'])[['Incidents Recorded']].sum().reset_index()

# Create a new dataframe merging the crime_df and crime_sum to include "Total incidents by Suburb/PC"
crime_final = pd.merge(crime_severity_added, crime_sev_sub_sum, how='outer',left_on=['SuburbPostcode','Rating'], right_on = ['SuburbPostcode','Rating'])
crime_final.rename(columns={'Incidents Recorded_x':'Incidents Recorded','Incidents Recorded_y':'Total Incidents by rating and Suburb/PC'}, inplace=True)

crime_final.head(2)

,Year,Year ending,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,SuburbPostcode,Total Incidents by Suburb/PC,Total Incidents by Offence Division and Suburb/PC,Total Incidents by Offence Subgroup and Suburb/PC,Rating,Total Incidents by rating and Suburb/PC
0,2019.0,December,Alpine,3691,Dederang,B Property and deception offences,B30 Burglary/Break and enter,B321 Residential non-aggravated burglary,1.0,"Dederang, 3691",59.0,20.0,2.0,0.0,6.0
1,2019.0,December,Alpine,3691,Dederang,B Property and deception offences,B20 Property damage,B21 Criminal damage,1.0,"Dederang, 3691",59.0,20.0,4.0,0.0,6.0


Create a list of offences to retain in the dataset so that they cover the possibile safety risks for someone who works out outdoors

In [15]:
retain_offences =  offence_severity_levels["Offence Subgroup"].unique().tolist()

Filter dataframe using the offence subgroups listed above

In [16]:
crime_severity_df = crime_final[crime_final["Offence Subgroup"].isin(retain_offences)]
crime_severity_df.head(2)

,Year,Year ending,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,SuburbPostcode,Total Incidents by Suburb/PC,Total Incidents by Offence Division and Suburb/PC,Total Incidents by Offence Subgroup and Suburb/PC,Rating,Total Incidents by rating and Suburb/PC
72592,2019.0,December,Alpine,3691,Dederang,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,1.0,"Dederang, 3691",59.0,12.0,4.0,3.0,5.0
72593,2018.0,December,Alpine,3691,Dederang,A Crimes against the person,A20 Assault and related offences,A232 Non-FV Common assault,1.0,"Dederang, 3691",59.0,12.0,2.0,3.0,5.0


Group the dataset by "SuburbPostcode" column to get the total number of retained offences of each "SuburbPostcode"

In [17]:
total_off_sub=crime_severity_df.groupby(['SuburbPostcode'])[['Incidents Recorded']].sum().reset_index()

# Create a new dataframe merging the crime_df and crime_sum to include "Total incidents by Suburb/PC"
total_off_sub_df = pd.merge(crime_severity_df, total_off_sub, how='outer',left_on=['SuburbPostcode'], right_on = ['SuburbPostcode'])
total_off_sub_df.rename(columns={'Incidents Recorded_x':'Incidents Recorded','Incidents Recorded_y':'Total Relevant Incidents - Suburb'}, inplace=True)

total_off_sub_df.head(2)

,Year,Year ending,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,SuburbPostcode,Total Incidents by Suburb/PC,Total Incidents by Offence Division and Suburb/PC,Total Incidents by Offence Subgroup and Suburb/PC,Rating,Total Incidents by rating and Suburb/PC,Total Relevant Incidents - Suburb
0,2019.0,December,Alpine,3691,Dederang,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,1.0,"Dederang, 3691",59.0,12.0,4.0,3.0,5.0,8.0
1,2018.0,December,Alpine,3691,Dederang,A Crimes against the person,A20 Assault and related offences,A232 Non-FV Common assault,1.0,"Dederang, 3691",59.0,12.0,2.0,3.0,5.0,8.0


----------------------------------------------------------------------------------

### Crime rate dataset

### Dataset Info

Name: Data Tables - LGA Criminal Incidents Visualisation - year ending December 2019

Description: Table 02 Criminal incidents and rate per 100,000 population by principal offence, local government area and police service area - January 2010 to December 2019

Link : https://www.crimestatistics.vic.gov.au/sites/default/files/embridge_cache/emshare/original/public/users/202003/e7/d73f8b607/Data_Tables_LGA_Recorded_Offences_Year_Ending_December_2019.xlsx

Permissions : CC BY 4.0

### Wrangling Tasks

Read data into python

In [18]:
rate_df = pd.read_excel('Data_Tables_LGA_Criminal_Incidents_Year_Ending_December_2019.xlsx','Table 02')
print("Dataset shape : ",rate_df.shape)
print(rate_df.columns)
rate_df.head(2)

Dataset shape :  (50104, 10)
Index(['Year', 'Year ending', 'Police Service Area', 'Local Government Area',
       'Offence Division', 'Offence Subdivision', 'Offence Subgroup',
       'Incidents Recorded', 'PSA Rate per 100,000 population',
       'LGA Rate per 100,000 population'],
      dtype='object')


,Year,Year ending,Police Service Area,Local Government Area,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,"PSA Rate per 100,000 population","LGA Rate per 100,000 population"
0,2019,December,Ballarat,Ballarat,A Crimes against the person,A10 Homicide and related offences,A10 Homicide and related offences,2,1.712247,1.827338
1,2019,December,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,150,128.418495,137.050315


Filter dataset for the relevant year range of 2017 -2019

In [19]:
rate_df=rate_df[rate_df["Year"].isin([2017, 2018,2019])]

Create new dataframe storing the median values of crimes in each LGA

In [20]:
median_rate = rate_df["LGA Rate per 100,000 population"].median()

Filter the dataframe to only includ ethe offences identified previously in the project

In [21]:
rate_df = rate_df[rate_df["Offence Subgroup"].isin(retain_offences)]
print(rate_df.shape)
rate_df.head(3)

(4128, 10)


,Year,Year ending,Police Service Area,Local Government Area,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,"PSA Rate per 100,000 population","LGA Rate per 100,000 population"
2,2019,December,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,169,144.684838,154.410021
5,2019,December,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A232 Non-FV Common assault,184,157.526687,168.115053
9,2019,December,Ballarat,Ballarat,A Crimes against the person,A50 Robbery,A51 Aggravated robbery,32,27.395946,29.237400


Calculate the meadian value of each offence in the above dataframe

In [22]:
lga_rate_medians = rate_df.groupby('Offence Subgroup')['LGA Rate per 100,000 population'].median().reset_index()
lga_rate_medians.head(2)

,Offence Subgroup,"LGA Rate per 100,000 population"
0,A212 Non-FV Serious assault,110.915310
1,A232 Non-FV Common assault,118.765584


Obtain total incidents recorded by suburb, offence and rating from total_off_sub_df and create a new dataset

In [23]:
incident_total = total_off_sub_df.groupby(['SuburbPostcode','Offence Subgroup','Rating'])[['Incidents Recorded']].sum().reset_index()
incident_total.head(2)

,SuburbPostcode,Offence Subgroup,Rating,Incidents Recorded
0,"Abbeyard, 3737",A212 Non-FV Serious assault,3.0,1.0
1,"Abbeyard, 3737",A232 Non-FV Common assault,3.0,1.0


Assign severity weights to each of the rating. 1 -> 0.2, 2 -> 0.3 and 3 -> 0.5, 

In [24]:
score_list = []

for i, row in incident_total.iterrows():
    if (incident_total['Rating'][i] == 3.0):
            score_list.append(0.5)
    elif (incident_total['Rating'][i] == 2.0):
            score_list.append(0.3)
    else:
        score_list.append(0.2)

incident_total['Severity Weight'] = pd.Series(score_list).values 

Create new dataframe to combine incident_total and lga_rate_medians through an outer join

In [25]:
median_comparison = pd.merge(incident_total, lga_rate_medians, how='outer',left_on=['Offence Subgroup'], right_on = ['Offence Subgroup'])
median_comparison.head(2)

,SuburbPostcode,Offence Subgroup,Rating,Incidents Recorded,Severity Weight,"LGA Rate per 100,000 population"
0,"Abbeyard, 3737",A212 Non-FV Serious assault,3.0,1.0,0.5,110.91531
1,"Abbotsford, 3067",A212 Non-FV Serious assault,3.0,69.0,0.5,110.91531


Check one suburb ("Beaconsfield, 3807") to inspect the dataset

In [26]:
median_comparison[median_comparison["SuburbPostcode"]=="Beaconsfield, 3807"]

,SuburbPostcode,Offence Subgroup,Rating,Incidents Recorded,Severity Weight,"LGA Rate per 100,000 population"
72,"Beaconsfield, 3807",A212 Non-FV Serious assault,3.0,20.0,0.5,110.915310
1182,"Beaconsfield, 3807",A232 Non-FV Common assault,3.0,26.0,0.5,118.765584
2190,"Beaconsfield, 3807",A51 Aggravated robbery,3.0,9.0,0.5,20.265544
2702,"Beaconsfield, 3807",A52 Non-Aggravated robbery,2.0,1.0,0.3,3.979991
2973,"Beaconsfield, 3807",A712 Non-FV Stalking,3.0,1.0,0.5,13.489831
3537,"Beaconsfield, 3807",A722 Non-FV Harassment and private nuisance,3.0,1.0,0.5,8.865490
4066,"Beaconsfield, 3807",A732 Non-FV Threatening behaviour,3.0,4.0,0.5,29.902518
5392,"Beaconsfield, 3807",A89 Other dangerous or negligent acts endanger...,1.0,5.0,0.2,38.437020
6425,"Beaconsfield, 3807",D11 Firearms offences,3.0,1.0,0.5,37.846532
7453,"Beaconsfield, 3807",D12 Prohibited and controlled weapons offences,3.0,23.0,0.5,86.003491



Create a new column in median_comparison dataset as "Flag" to store a binary ndicator that denotes whether the incidents recorded in that specific suburb is greater than the LGA Rate per 100,000 population. Create another column "Suburb Score" to store the multiplication of flag and the severity weight

In [27]:
median_comparison['Flag'] = np.where(median_comparison['Incidents Recorded']>median_comparison['LGA Rate per 100,000 population'], 1,0)
median_comparison['Suburb Score'] = median_comparison['Flag']*median_comparison['Severity Weight']
median_comparison.head(2)

,SuburbPostcode,Offence Subgroup,Rating,Incidents Recorded,Severity Weight,"LGA Rate per 100,000 population",Flag,Suburb Score
0,"Abbeyard, 3737",A212 Non-FV Serious assault,3.0,1.0,0.5,110.91531,0,0.0
1,"Abbotsford, 3067",A212 Non-FV Serious assault,3.0,69.0,0.5,110.91531,0,0.0


Group the median comparison dataframe by "SuburbPostcode" column to obtain the total Suburb Score

In [28]:
median_comparison_df =median_comparison.groupby(['SuburbPostcode'])[['Suburb Score']].sum().reset_index()
median_comparison_df.head(2)

,SuburbPostcode,Suburb Score
0,"Abbeyard, 3737",0.0
1,"Abbotsford, 3067",1.6


Create column "Indicator" to assign "Safe" and "Unsafe" values to each suburb based on the total suburb score. If suburb score = 0, the indicator is "Safe" and if suburb score >0, the indicator is "Unsafe"

In [29]:
median_comparison_df['Indicator'] = np.where(median_comparison_df['Suburb Score']==0, "Safe","Unsafe")
median_comparison_df.rename(columns={'Local Government Area':'LGA','Suburb, Postcode':'SuburbPostcode','Suburb Score':'SuburbScore' }, inplace=True)

median_comparison_df.head(2)

,SuburbPostcode,SuburbScore,Indicator
0,"Abbeyard, 3737",0.0,Safe
1,"Abbotsford, 3067",1.6,Unsafe


Merge the median_comparison_df with lga_suburb so that the missing values can be addressed

In [30]:
mc_df = lga_suburb.merge(median_comparison_df, on='SuburbPostcode', how='left')
mc_df=mc_df.drop(['LGA'], axis = 1)

lga_suburb= lga_suburb.drop_duplicates()

Count duplicates in the above dataframe

In [31]:
len(mc_df[mc_df['SuburbPostcode'].duplicated() & mc_df['SuburbScore'].duplicated()])

256

Drop duplicates and prpare yej dayaset for export. Missing valuesin the "SuburubScore" and "Indicator" fields are also addressed in the following code

In [32]:
mc_df["SuburbScore"].fillna('Score Unavailable', inplace=True)
mc_df["Indicator"].fillna('Data Unavailable', inplace=True)
mc_df= mc_df.drop_duplicates()
mc_df.head(2)

,SuburbPostcode,SuburbScore,Indicator
0,"Dederang, 3691",0,Safe
1,"Glen Creek, 3691",0,Safe


Export dataset as CSV file

In [33]:
mc_df.to_csv("safety_score_export.csv", index=False)

-------------------

## Crime Location Dataset

### Dataset Info

Name: Data Tables - LGA Criminal Incidents Visualisation - year ending December 2019

Description: Table 04 Criminal incidents by location type and local government area - January 2010 to December 2019

Link : https://www.crimestatistics.vic.gov.au/sites/default/files/embridge_cache/emshare/original/public/users/202003/e7/d73f8b607/Data_Tables_LGA_Recorded_Offences_Year_Ending_December_2019.xlsx

Permissions : CC BY 4.0


### Wrangling Tasks

Read dataset to Python

In [34]:
crime_loc_df = pd.read_excel('Data_Tables_LGA_Criminal_Incidents_Year_Ending_December_2019.xlsx','Table 04')
print("Dataset shape : ",crime_loc_df.shape)
print(crime_loc_df.columns)
crime_loc_df.head(2)

Dataset shape :  (52614, 7)
Index(['Year', 'Year ending', 'Local Government Area', 'Location Division',
       'Location Subdivision', 'Location Group', 'Incidents Recorded'],
      dtype='object')


,Year,Year ending,Local Government Area,Location Division,Location Subdivision,Location Group,Incidents Recorded
0,2019,December,Alpine,1 Residential,11 Dwelling - private,111 Flat/Unit/Apartment,1
1,2019,December,Alpine,1 Residential,11 Dwelling - private,112 House,123


Filter dataset to reflect the 2017-2019 date range

In [35]:
crime_loc_df=crime_loc_df[crime_loc_df["Year"].isin([2017, 2018,2019])]
crime_loc_df.shape

(16566, 7)

Drop unnecessary columns

In [36]:
drop_columns = ["Year","Year ending","Location Division","Location Subdivision"]
crime_loc_df=crime_loc_df.drop(drop_columns, axis = 1)

Group crimes by LGA

In [37]:
lga_incidents =crime_loc_df.groupby(['Local Government Area'])[['Incidents Recorded']].sum().reset_index()

lga_crime_loc = pd.merge(crime_loc_df, lga_incidents, how='outer',left_on=['Local Government Area'], right_on = ['Local Government Area'])
lga_crime_loc.rename(columns={'Incidents Recorded_x':'Incidents Recorded','Incidents Recorded_y':'Total LGA Incidents'}, inplace=True)

lga_crime_loc.head(2)

,Local Government Area,Location Group,Incidents Recorded,Total LGA Incidents
0,Alpine,111 Flat/Unit/Apartment,1,1094
1,Alpine,112 House,123,1094


Filter crimes for the following locations
1.       Parkland/Reserve
2.       Vacant Block
3.       Beach Foreshore
4.       Other Open Space
5.       Street/Lane/Footpath
6.       Sport Area/Facility
7.       Other Rec./Sports


In [38]:
location_groups =["271 Parkland/Reserve", "273 Vacant Block", "274 Beach Foreshore","275 Other Open Space","281 Street/Lane/Footpath", "384 Sport Area/Facility","386 Other Rec./Sports"]
lga_crime_loc=lga_crime_loc[lga_crime_loc["Location Group"].isin(location_groups)]
print(lga_crime_loc.shape)
lga_crime_loc[lga_crime_loc["Local Government Area"]=="Alpine"].head(2)

(1418, 4)


,Local Government Area,Location Group,Incidents Recorded,Total LGA Incidents
22,Alpine,271 Parkland/Reserve,12,1094
24,Alpine,273 Vacant Block,1,1094


Include total incidents in each LGA and totals for each offence location group

In [39]:
lga_loc_group=lga_crime_loc.groupby(['Local Government Area','Location Group'])[['Incidents Recorded']].sum().reset_index()

lga_loc_crime = pd.merge(lga_crime_loc, lga_loc_group, how='outer',left_on=['Local Government Area','Location Group'], right_on = ['Local Government Area','Location Group'])
lga_loc_crime.rename(columns={'Incidents Recorded_x':'Incidents Recorded','Incidents Recorded_y':'LGA Location Total'}, inplace=True)

print(len(lga_loc_crime))
lga_loc_crime[lga_loc_crime["Local Government Area"]=="Alpine"].head(2)

1418


,Local Government Area,Location Group,Incidents Recorded,Total LGA Incidents,LGA Location Total
0,Alpine,271 Parkland/Reserve,12,1094,50
1,Alpine,271 Parkland/Reserve,26,1094,50


Get the total LGA incidents

In [40]:
total_lga_incidents = (lga_loc_crime[["Local Government Area","Total LGA Incidents","Incidents Recorded"]]).groupby(["Local Government Area"])[["Total LGA Incidents"]].mean().reset_index()
total_lga_incidents.head(2)

,Local Government Area,Total LGA Incidents
0,Alpine,1094
1,Ararat,2573


Merge the two dataframes lga_loc_group and total_lga_incidents

In [41]:
final_loc_df = pd.merge(lga_loc_group, total_lga_incidents, on='Local Government Area', how='inner')
location= final_loc_df["Location Group"].str.split(" ", n = 1, expand = True) 
final_loc_df["Location"] = location[1]
final_loc_df.head(2)

,Local Government Area,Location Group,Incidents Recorded,Total LGA Incidents,Location
0,Alpine,271 Parkland/Reserve,50,1094,Parkland/Reserve
1,Alpine,273 Vacant Block,3,1094,Vacant Block


Create the final dataframe to export by making the necessary formatting changes

In [42]:
final_loc_df["Percentage"] =((final_loc_df["Incidents Recorded"] /final_loc_df["Total LGA Incidents"])*100)
location= final_loc_df["Location Group"].str.split(" ", n = 1, expand = True) 
final_loc_df["Location"] = location[1]

drop_columns = ["Location Group","Incidents Recorded","Total LGA Incidents"]
final_loc_df=final_loc_df.drop(drop_columns, axis = 1)
final_loc_df.rename(columns={'Local Government Area':'LGA'}, inplace=True)

final_loc_df.head(2)

,LGA,Location,Percentage
0,Alpine,Parkland/Reserve,4.570384
1,Alpine,Vacant Block,0.274223


Export dataset as a CSV file

In [43]:
final_loc_df.to_csv("lga_crimes_export.csv", index=False)

------------------------------------------------------------------------------

#### Export files produced

* lga_suburb_export--> Contains the LGA, Suburb and Postcode information

* safety_score_export -->  Contains the safety score and Indicator for each Suburb-Postcode

* lga_crimes_export.csv --> Contains the LGA, Location and percentage of crimes


---------------------------------------------

## End of data wrangling process for iteration 01